In [1]:
import re
import os
import json
import pandas as pd

In [2]:
with open(os.path.join("data_fixes", "opravy.json"), encoding="utf-8") as opravy:
    opravy = json.loads(opravy.read())

In [3]:
persony = pd.read_json(os.path.join("data", "persony_raw.json"))

In [4]:
persony

,Jméno,Id,Medailon,Osoba,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí,Zasloužilý umělec,Národní umělec
0,Ondřej Mašek (1000),1000,,fyzická,None,None,None,None,None,None,NaN,NaN
1,kino Světozor (100019),100019,None,právnická,None,None,None,None,None,None,NaN,NaN
2,"AD WOOD, s.r.o. (100038)",100038,None,právnická,None,None,None,None,None,None,NaN,NaN
3,Hypermarket Film s.r.o. (100039),100039,None,právnická,Hypermarket Film,None,None,None,None,None,NaN,NaN
4,Hypermarket Film s.r.o. (100039),100039,None,právnická,Hypermarket Film,None,None,None,None,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
35810,Josef Dvořák (9998),9998,"herec, divadelní podnikatel",fyzická,Josef Voloďa Dvořák (rodné jméno),None,25.04.1942,"Horní Cerekev, Československo",None,None,NaN,NaN
35811,Antonín Dimitrov (99996),99996,"výtvarník, grafik, scénograf, designér",fyzická,None,None,27.02.1928,"Mšecké Žehrovice, Československo",27.12.2014,"Bobcaygeon, Kanada",NaN,NaN
35812,Čechomor (person),person,None,právnická,None,None,None,None,None,None,NaN,NaN
35813,Kevin Mc Kidd (person),person,,fyzická,None,None,None,None,None,None,NaN,NaN


In [5]:
def odnebovat(x):
    if x:
        if "nebo" in x:
            x = x.split("nebo")[1].strip()
    return x

In [6]:
def rozdel_misto(x):
    if type(x) == str:
        try:
            mesto = x.split(",")[0]
            stat = x.split(",")[1]
        except:
            mesto = None
            stat = x
    else:
        mesto = None
        stat = None
    return [mesto, stat]

## Doplnění dat z Wikidat

In [7]:
persony[persony["Jméno"] == "Stanislav Litera (35280)"].sample(1)

,Jméno,Id,Medailon,Osoba,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí,Zasloužilý umělec,Národní umělec
18963,Stanislav Litera (35280),35280,,fyzická,None,None,None,None,None,None,NaN,NaN


In [8]:
with open(os.path.join("data_fixes", "data_z_wikidat.json"), encoding="utf-8") as data_z_wikidat:
    data_z_wikidat = json.loads(data_z_wikidat.read())

In [9]:
for key, val in data_z_wikidat.items():
    persony.loc[persony["Jméno"] == key, "Datum narození"] = val[0]

In [10]:
for key, val in data_z_wikidat.items():
    persony.loc[persony["Jméno"] == key, "Datum úmrtí"] = val[1]

In [11]:
persony[persony["Jméno"] == "Stanislav Litera (35280)"].sample(1)

,Jméno,Id,Medailon,Osoba,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí,Zasloužilý umělec,Národní umělec
18963,Stanislav Litera (35280),35280,,fyzická,None,None,11.06.1931,None,21.09.2003,None,NaN,NaN


## Opravy

In [12]:
persony[persony["Jméno"] == "Eva Pecková (34563)"]

,Jméno,Id,Medailon,Osoba,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí,Zasloužilý umělec,Národní umělec
18566,Eva Pecková (34563),34563,"pedagožka, zpěvačka, tanečnice",fyzická,Eva Kubičková (rodné jméno),None,09.12.9441,"Volyně, Československo",18.07.2018,"České Budějovice, Česká republika",NaN,NaN


In [13]:
for key, val in opravy["data_narozeni"].items():
    persony.loc[persony["Jméno"] == key, "Datum narození"] = val

In [14]:
for key, val in opravy["jmena"].items():
    persony.loc[persony["Jméno"] == key, "Jméno"] = val

In [15]:
persony[persony["Jméno"] == "Eva Pecková (34563)"]

,Jméno,Id,Medailon,Osoba,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí,Zasloužilý umělec,Národní umělec
18566,Eva Pecková (34563),34563,"pedagožka, zpěvačka, tanečnice",fyzická,Eva Kubičková (rodné jméno),None,,"Volyně, Československo",18.07.2018,"České Budějovice, Česká republika",NaN,NaN


In [16]:
persony[persony["Jméno"] == "Adam Klemens (1779)"]

,Jméno,Id,Medailon,Osoba,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí,Zasloužilý umělec,Národní umělec
11585,Adam Klemens (1779),1779,"dirigent, hudební skladatel",fyzická,None,None,14.01.1967,None,None,None,NaN,NaN


## Konverze formátů

In [17]:
persony["Datum narození"].sample(200).unique()

array([None, '1963', '04.07.1977', '12.09.1957', '28.05.1925',
       '26.03.1983', '15.04.1915', '30.03.1978', '1981', '13.11.1812',
       '1983', '1947', '07.06.1974', '07.10.1923', '05.02.1934',
       '28.03.1980', '26.09.1970', '28.07.1928', '16.01.1986',
       '18.10.1933', '1974', '04.07.1970', '27.05.1949', '1946',
       '12.03.1957', '23.02.1932', '21.08.1871', '09.12.1934',
       '08.01.1947', '23.04.1941', '1955', '06.12.1924', '09.02.1927',
       '28.02.1943', '1931', '14.05.1863', '17.12.1948', '15.05.1984',
       '15.06.1932', '28.08.1884', '1968', '09.03.1916', '1930',
       '17.10.1903', '24.02.1897', '31.07.1971', '05.12.1950',
       '05.08.1964', '06.08.1994', '07.05.1896', '30.03.1946',
       '10.08.1925', '4. 8. 1958', '25.03.1966', '26.11.1895',
       '30.10.1907', '31.01.1970', '14.12.1940', '02.02.1963',
       '30.08.1879', '08.04.1966', '27.07.1948', '13.07.1937',
       '17.05.1950', '14.10.1882', '16.11.1916', '11.04.1944',
       '22.11.1986', '24.

In [18]:
persony["Město narození"] = persony["Místo narození"].apply(lambda x: rozdel_misto(x)[0])
persony["Stát narození"] = persony["Místo narození"].apply(lambda x: rozdel_misto(x)[1])
persony["Město úmrtí"] = persony["Místo úmrtí"].apply(lambda x: rozdel_misto(x)[1])
persony["Stát úmrtí"] = persony["Místo úmrtí"].apply(lambda x: rozdel_misto(x)[1])
persony["Rok narození"] = pd.to_numeric(persony["Datum narození"].apply(lambda x: x[-4:] if isinstance(x, str) else None), errors="coerce")
persony["Rok úmrtí"] = pd.to_numeric(persony["Datum úmrtí"].apply(lambda x: x[-4:] if isinstance(x, str) else None), errors="coerce")

In [19]:
def na_data(sloupec):
    global persony
    persony["Pracovni"] = persony[sloupec]
    persony["Pracovni"] = persony["Pracovni"].apply(lambda x: odnebovat(x))
    for index, row in persony.iterrows():
        try:
            date = pd.to_datetime(row["Pracovni"], format = "%d.%m.%Y", errors="ignore")
        except ValueError:
            date = pd.to_datetime(row["Pracovni"], format = "%Y", errors="ignore")
        except ValueError:
            date = None
        persony.at[index, sloupec] = date
    persony = persony.drop(["Pracovni"], axis = 1)

In [20]:
na_data("Datum narození")
na_data("Datum úmrtí")

In [21]:
persony = persony.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [22]:
persony

,Jméno,Id,Medailon,Osoba,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí,Zasloužilý umělec,Národní umělec,Město narození,Stát narození,Město úmrtí,Stát úmrtí,Rok narození,Rok úmrtí
0,Ondřej Mašek (1000),1000,,fyzická,None,None,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN
1,kino Světozor (100019),100019,None,právnická,None,None,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN
2,"AD WOOD, s.r.o. (100038)",100038,None,právnická,None,None,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN
3,Hypermarket Film s.r.o. (100039),100039,None,právnická,Hypermarket Film,None,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN
4,Hypermarket Film s.r.o. (100039),100039,None,právnická,Hypermarket Film,None,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35810,Josef Dvořák (9998),9998,"herec, divadelní podnikatel",fyzická,Josef Voloďa Dvořák (rodné jméno),None,1942-04-25 00:00:00,"Horní Cerekev, Československo",None,None,NaN,NaN,Horní Cerekev,Československo,None,None,1942.0,NaN
35811,Antonín Dimitrov (99996),99996,"výtvarník, grafik, scénograf, designér",fyzická,None,None,1928-02-27 00:00:00,"Mšecké Žehrovice, Československo",2014-12-27 00:00:00,"Bobcaygeon, Kanada",NaN,NaN,Mšecké Žehrovice,Československo,Kanada,Kanada,1928.0,2014.0
35812,Čechomor (person),person,None,právnická,None,None,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN
35813,Kevin Mc Kidd (person),person,,fyzická,None,None,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN


In [23]:
persony.to_json(os.path.join("data", "persony.json"), orient="records")